In [1]:
import numpy as np
import pandas as pd

from typing import Dict

import pyspark
from pyspark import StorageLevel
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)
from pyspark.sql.functions import (
    col,
    isnan,
    when,
    count,
)
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    OneHotEncoderModel,
    StringIndexerModel,
    VectorAssembler,
    ImputerModel,
)
from pyspark.ml.classification import (
    LogisticRegressionModel,
)
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator,
    MulticlassClassificationEvaluator,
)

import itertools

import pickle

import os
os.chdir("/home/aap2239/interpretable-credit-default-prediction")

from icdp_predictor import *

In [2]:
spark = get_spark_session()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/16 08:49:05 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/12/16 08:49:05 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/12/16 08:49:05 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/12/16 08:49:05 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
meta_data = load_meta_data()

In [4]:
models_dict = load_models(spark = spark)

22/12/16 08:49:17 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)


In [5]:
models_dict

{'indexer_model': StringIndexerModel: uid=StringIndexer_f63dd7cda7d2, handleInvalid=keep, numInputCols=10, numOutputCols=10,
 'imputer_model': ImputerModel: uid=Imputer_b3b5a27f34d8, strategy=median, missingValue=NaN, numInputCols=138, numOutputCols=138,
 'ohe_model': OneHotEncoderModel: uid=OneHotEncoder_1f05db89aeb8, dropLast=true, handleInvalid=error, numInputCols=10, numOutputCols=10,
 'va_model': VectorAssembler_cba0b0e097b9,
 'lr_model': LogisticRegressionModel: uid=LogisticRegression_111045ad469c, numClasses=2, numFeatures=477}

In [8]:
preds = pipeline_pred(
    path = 'gs://icdp-bigdata-bucket/train_data.csv',
    limit_of_df = 100000,
    spark = spark,
    models_dict = models_dict,
    meta_data = meta_data,
)

In [9]:
preds.show(50)

+--------------------+--------------------+--------------------+----------+
|         customer_ID|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|000041bdba6ecadd8...|[3.52528436018695...|[0.97139868721661...|       0.0|
|000084e5023181993...|[4.54615530169273...|[0.98950343630857...|       0.0|
|0001337ded4e1c253...|[-1.5594244411381...|[0.17372925148198...|       1.0|
|000201146e53cacdd...|[3.61952868512074...|[0.97390394903888...|       0.0|
|000391f219520dbca...|[-4.2262432557652...|[0.01439686561392...|       1.0|
|000473eb907b57c8c...|[-0.8457904409469...|[0.30031665348842...|       1.0|
|0004e00358fc4dd63...|[5.84402947988304...|[0.99711122411655...|       0.0|
|0004ec03ca1ab2adb...|[5.95678765067497...|[0.99741846867653...|       0.0|
|00050d84c6d26e26c...|[4.72075805885786...|[0.99117023638887...|       0.0|
|00055add5eaee481b...|[4.71772773441095...|[0.99114367608659...|       0.0|
|0005e52a3fa